<a href="https://colab.research.google.com/github/jonnyderme/Multimedia-Systems/blob/main/Auth_Multimedia_Systems_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries that may be needed**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import pandas as pd

from google.colab.patches import cv2_imshow
from PIL import Image
from scipy.fft import dct
from scipy.fft import idct
from scipy.sparse import spdiags


# **3.1 Preprocess**

In [ ]:
def convert2ycbcr(imageRGB, subimg):

    # Check input validity
    if (len(imageRGB.shape) != 3 and imageRGB.shape[2] != 3) :
        raise ValueError("Error. 1st argument 'imageRGB' must be a 3d matrix.")
    if subimg.shape != (3,):
        raise ValueError("Error. 2nd argument 'subimg' must be a 1x3 vector.")

    # Ensure image dimensions are multiples of 8
    N, M, _ = imageRGB.shape
    Nmod8 = N % 8
    Mmod8 = M % 8

    # Crop the border lines and columns if image dimensions are not multiples of 8
    if Nmod8 != 0:
       NcropStart = math.floor(Nmod8/2)
       NcropEnd = round(Nmod8/2)
       imageRGB = imageRGB[NcropStart : -NcropEnd, :, :]
    if Mmod8 != 0:
       McropStart = math.floor(Mmod8/2)
       McropEnd = round(Mmod8/2)
       imageRGB = imageRGB[ :, McropStart : -McropEnd, :]

    # Initialize transformation matrix T
    T = np.array([ [0.299,     0.587,     0.114 ]   ,
                   [-0.1687,  -0.3313,    0.5   ]   ,
                   [0.5,      -0.4187,   -0.0813]  ])

    # Get the individuals colors from the RGB image and transform to YCbCr
    RowNumber, ColumnNumber, _ = imageRGB.shape
    red   = imageRGB[:, :, 0].reshape(-1,1)
    green = imageRGB[:, :, 1].reshape(-1,1)
    blue  = imageRGB[:, :, 2].reshape(-1,1)

    colours = np.concatenate((red, green, blue), axis=1)

    transformed = np.dot(colours.astype(np.double), np.transpose(T)) + [0, 128, 128]
    transformed = np.round(transformed)
    transformed = transformed.astype(np.uint8)

    imageY =  np.uint8(transformed[:, 0].reshape(ColumnNumber, RowNumber))
    imageCb = np.uint8(transformed[:, 1].reshape(ColumnNumber, RowNumber))
    imageCr = np.uint8(transformed[:, 2].reshape(ColumnNumber, RowNumber))

    # Subsampling
    if np.array_equal(subimg, [4, 4, 4]):
        pass

    elif np.array_equal(subimg, [4, 2, 2]):
        imageCb = imageCb[:, ::2]
        imageCr = imageCr[:, ::2]

    elif np.array_equal(subimg, [4, 2, 0]):
        imageCb = imageCb[::2, ::2]
        imageCr = imageCr[::2, ::2]

    else:
        raise ValueError("Error. The provided subsampling rate is not supported.")

    return imageY, imageCb, imageCr

In [ ]:
def convert2rgb(imageY, imageCr, imageCb, subimg):

    # Check input validity
    if subimg.shape != (3,):
        raise ValueError("Error. 4th argument 'subimg' must be a 1x3 vector.")

    # Initialize transformation matrix T and its inverse
    T = np.array([   [0.299,    0.587,    0.114],
                     [-0.1687, -0.3313,   0.5],
                     [0.5,     -0.4187,  -0.0813] ])
    invT = np.linalg.inv(T)

    # Subsampling
    RowNum, ColumnNum = imageY.shape
    imageRGB = np.zeros((RowNum, ColumnNum, 3), dtype=np.uint8)
    imageCr_rec = np.zeros((RowNum, ColumnNum), dtype=np.uint8)
    imageCb_rec = np.zeros((RowNum, ColumnNum), dtype=np.uint8)

    if   np.array_equal(subimg, [4, 4, 4]):
         imageCb_rec = imageCb
         imageCr_rec = imageCr

    elif np.array_equal(subimg, [4, 2, 2]):
         imageCb_rec[:, ::2] = imageCb
         imageCr_rec[:, ::2] = imageCr
         imageCb_rec[:, 1::2] = imageCb
         imageCr_rec[:, 1::2] = imageCr

    elif np.array_equal(subimg, [4, 2, 0]):
         imageCb_rec[::2, ::2] = imageCb
         imageCr_rec[::2, ::2] = imageCr
         imageCb_rec[1::2, ::2] = imageCb
         imageCr_rec[1::2, ::2] = imageCr
         imageCb_rec[::2, 1::2] = imageCb
         imageCr_rec[::2, 1::2] = imageCr
         imageCb_rec[1::2, 1::2] = imageCb
         imageCr_rec[1::2, 1::2] = imageCr

    else:
        raise ValueError("Error. The provided subsampling rate doesn't exist.")

    # Transform YCbCr to RGB
    Y  =  np.transpose(imageY.flatten())
    Cb =  np.transpose(imageCb_rec.flatten())
    Cr =  np.transpose(imageCr_rec.flatten())

    YCbCr = np.stack((Y, Cb, Cr), axis=1)

    itransformed = (YCbCr - [0, 128, 128]) @ np.transpose(invT)
    itransformed = np.round(itransformed)
    itransformed = itransformed.astype(np.uint8)

    # Extract RGB channels
    red   = np.uint8(itransformed[:, 0].reshape(ColumnNum, RowNum))
    green = np.uint8(itransformed[:, 1].reshape(ColumnNum, RowNum))
    blue  = np.uint8(itransformed[:, 2].reshape(ColumnNum, RowNum))

    # Assign RGB channels to the output image
    imageRGB[:, :, 0] = red
    imageRGB[:, :, 1] = green
    imageRGB[:, :, 2] = blue

    return imageRGB

# **3.2 DCT Transformation**

In [ ]:
def blockDCT(block):
    if block.shape != (8, 8):
        raise ValueError("Error. The argument {block} is not an 8x8 matrix.")

    # Offset the block values by 2^(8-1) (8 bit color resolution)
    blockOffset = np.float32(block) - 2**(8-1)

    # Compute the DCT coefficients for this block
    dctBlock = dct(dct(blockOffset, axis=0, norm='ortho'), axis=1, norm='ortho')
    return dctBlock

In [ ]:
def iBlockDCT(dctBlock):

    if dctBlock.shape != (8, 8):
        raise ValueError("Error. The argument {dctBlock} is not an 8x8 matrix.")

    # Compute the inverse DCT to get the block values
    blockValues = idct(idct(dctBlock, axis=0, norm='ortho'), axis=1, norm='ortho')

    # Undo the offset of the block values by 2^(8-1) (8 bit color resolution)
    block = np.float64(blockValues + 2**(8-1))

    return block

# **3.3 Quantization**

In [ ]:
def quantizeJPEG(dctBlock, qTable, qScale):

    if dctBlock.shape != (8, 8):
        raise ValueError("Error. 1st argument {dctBlock} is not a 8x8 matrix.")

    if qTable.shape != (8, 8):
        raise ValueError("Error. 2nd argument {qTable} is not a 8x8 matrix.")

    if not np.isscalar(qScale):
        raise ValueError("Error. 3rd argument {qScale} is not scalar.")

    # Perform point-wise division of DCT coefficients by (qScale * qTable) and round to nearest integer
    qBlock = np.round(dctBlock / (qScale * qTable))

    return qBlock

In [ ]:
def dequantizeJPEG(qBlock, qTable, qScale):

    if qBlock.shape != (8, 8):
        raise ValueError("Error. 1st argument {qBlock} is not a 8x8 matrix.")

    if qTable.shape != (8, 8):
        raise ValueError("Error. 2nd argument {qTable} is not a 8x8 matrix.")

    if not np.isscalar(qScale):
        raise ValueError("Error. 3rd argument {qScale} is not scalar.")

    # Undo quantization by performing point-wise multiplication
    dctBlock = qBlock * (qScale * qTable)

    return dctBlock

In [ ]:
def changedTables(num):
    # Define the original tables
    qTableL = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                        [12, 12, 14, 19, 26, 58, 60, 55],
                        [14, 13, 16, 24, 40, 57, 69, 56],
                        [14, 17, 22, 29, 51, 87, 80, 62],
                        [18, 22, 37, 56, 68, 109, 103, 77],
                        [24, 35, 55, 64, 81, 104, 113, 92],
                        [49, 64, 78, 87, 103, 121, 120, 101],
                        [72, 92, 95, 98, 112, 100, 103, 99]])

    qTableC = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                        [18, 21, 26, 66, 99, 99, 99, 99],
                        [24, 26, 56, 99, 99, 99, 99, 99],
                        [47, 66, 99, 99, 99, 99, 99, 99],
                        [99, 99, 99, 99, 99, 99, 99, 99],
                        [99, 99, 99, 99, 99, 99, 99, 99],
                        [99, 99, 99, 99, 99, 99, 99, 99],
                        [99, 99, 99, 99, 99, 99, 99, 99]])

    # Check if the length of the arrays match
    if qTableL.shape[1] != qTableC.shape[1]:
        raise ValueError("Length of arrays doesn't match")

    # If num = 0, return the original tables
    if num == 0:
        return qTableL, qTableC

    rowNum = np.floor(num/qTableL.shape[1])
    rowNum = rowNum.astype(int)

    for i in range(rowNum+1) :
         if i < rowNum :
            qTableL[qTableL.shape[0]-1-i,:] = 1000
            qTableC[qTableC.shape[0]-1-i,:] = 1000
         else:
            lastElements = num -  rowNum * qTableL.shape[1]
            if lastElements != 0:
               qTableL[qTableC.shape[0]-1-i, -lastElements:] = 1000
               qTableC[qTableC.shape[0]-1-i, -lastElements:] = 1000

    return qTableL, qTableC

# **3.4 Zig-zag scanning and RLE**

In [ ]:
def runLength(qBlock, DCpred):
    if qBlock.shape != (8, 8):
        raise ValueError("Error. 1st argument qBlock must be a 8x8 matrix.")
    if not np.isscalar(DCpred):
        raise ValueError("Error. 2nd argument DCpred must be scalar.")

    # DC coefficient has different handling
    DC_coef = qBlock[0, 0] - DCpred

    # ZigZag re-ordering of the matrix
    # Create indexes
    idx = ( np.arange(qBlock.size).reshape(qBlock.shape) + 1)
    idx = np.flipud(idx)
    idx = np.fliplr(idx)

    # Number of additional diagonals needed
    numDiagonals = idx.shape[1] - 1

    # Initialize the diagonals and their positions
    diagonals = [idx[:, i] for i in range(idx.shape[1])]
    positions = np.arange(0, -numDiagonals-1, -1)

    # Create the sparse matrix using spdiags
    idxExtended = spdiags(diagonals, positions, idx.shape[0] + numDiagonals, idx.shape[1] + numDiagonals).toarray()
    idxExtendedT = np.transpose(idxExtended.astype(int))

    # Find rows where all elements are zero
    idxRows = np.any(idxExtendedT != 0, axis=1)

    # Select non-zero rows
    idx = idxExtendedT[idxRows]
    idx = np.fliplr(idx)
    # Reverse order of odd columns
    idx[:, 0::2] = np.flipud(idx[:, 0::2])
    # Flatten the array and discard zero-indexes
    idx = np.transpose(idx)
    idx = idx[idx != 0]
    idx = idx - 1

    qBlockT = np.transpose(qBlock)
    qBlockT = qBlockT.flatten()

    # Re-order array based on the ziz-zag indexes
    qBlock = qBlockT[idx]

    runSymbols = np.array([[0, DC_coef]])
    precedingZeros = 0

    # For all the AC coefficients
    for i in range(1, len(qBlock)):
        if qBlock[i] != 0:   #  Next Symbol Found, Update runSymbols
            runSymbols = np.vstack((runSymbols, [precedingZeros, qBlock[i]]))
            precedingZeros = 0      # Reset the procedure
        elif i == len(qBlock) - 1:  # EOB and qBlock(end) == 0
            while np.array_equal(runSymbols[-1], [15, 0]):   # Check if [15 0] was inserted at the end
                runSymbols = runSymbols[:-1]      # If that's the case, delete them to place [0 0]
            runSymbols = np.vstack((runSymbols, [0, 0]))  # Remaining symbols is 0
        else:
            precedingZeros += 1       # Count the preceding zeros
            if precedingZeros == 16:  # At 16, reset and add [15 0]
                runSymbols = np.vstack((runSymbols, [15, 0]))   # Runlength supports only 16 consecutively zeros
                precedingZeros = 0

    return runSymbols

In [ ]:
def irunLength(runSymbols, DCpred):
    runSymbols = np.array(runSymbols)  # Convert runSymbols to a numpy array

    if runSymbols.shape[1] != 2:
        raise ValueError("Error. 1st argument {runSymbols} must be a Rx2 matrix.")
    if not np.isscalar(DCpred):
        raise ValueError("Error. 2nd argument {DCpred} must be scalar.")

    qBlock = np.zeros((1, 1))
    qBlock[0, 0] = runSymbols[0, 1] + DCpred

    for i in range(1, runSymbols.shape[0]):  # For each row, minus DC coefficient
        if np.array_equal(runSymbols[i,:], [15, 0]):
            qBlock = np.hstack((qBlock, np.zeros((1, 16))))  # Place the 16 zeros
        elif np.array_equal(runSymbols[i,:], [0, 0]):
            qBlock = np.hstack((qBlock, np.zeros((1, 64 - len(qBlock)))))  # The rest is zeros
        else:
            preceding_zeros = np.zeros((1, runSymbols[i, 0]))
            quant_symbol = np.full((1, 1), runSymbols[i, 1])
            qBlock = np.hstack((qBlock, np.hstack((preceding_zeros, quant_symbol))))

    # Undo the ZigZag reorder
    idx = np.array([[0, 1, 5, 6, 14, 15, 27, 28],
                    [2, 4, 7, 13, 16, 26, 29, 42],
                    [3, 8, 12, 17, 25, 30, 41, 43],
                    [9, 11, 18, 24, 31, 40, 44, 53],
                    [10, 19, 23, 32, 39, 45, 52, 54],
                    [20, 22, 33, 38, 46, 51, 55, 60],
                    [21, 34, 37, 47, 50, 56, 59, 61],
                    [35, 36, 48, 49, 57, 58, 62, 63]])
    idxF = idx
    qBlockI = qBlock[0,idxF]

    return qBlockI

# **3.5 Huffman Coding**

## **Αuxiliary Functions**

In [ ]:
def getBinary(num, category):
    num = num.astype(int)
    # Based on Category, produce the binary / huffman value of num
    if not category:
        return '0'  # If category = 0, then binary value is zero too.

    binaryValue = [int(bit) for bit in format(abs(num), 'b').zfill(category)]

    if num < 0:
        binaryValue = [1 - bit for bit in binaryValue]  # For negative values, compute the 1's complement.

    binaryString = ''.join(map(str, binaryValue))  # Convert the list of bits to a string.

    # Truncate to match category length
    if len(binaryString) > category:
        binaryString = binaryString[:category]

    return binaryString

In [ ]:
def getDecimal(currentChar):
    # Based on Category, invert the Huffman code
    if currentChar[0] == '0':  # Negative values begin with 0
       currentChar = [1 - int(bit) for bit in currentChar]  # Compute the 1's complement
       decimalValue = -int(''.join(map(str, currentChar)), 2)  # Convert binary to decimal and make it negative
    else:
        decimalValue = int(currentChar, 2)  # For positive values, compute the decimal value of the binary

    return decimalValue

## **Huffman Encoder**

In [ ]:
def huffEnc(runSymbols, DC_Huffman_L, DC_Huffman_C, AC_Huffman_L, AC_Huffman_C, isLuminance):
    # Check input validity
    if not isinstance(runSymbols, np.ndarray) or runSymbols.shape[1] != 2:
        raise ValueError('Error. The argument {runSymbols} must be a Rx2 matrix.')

    # Get the number of rows in the runSymbols array
    RowNumber = runSymbols.shape[0]
    # Initialize an empty string to store the Huffman-encoded stream
    strHuff = ""

    # Select Huffman tables based on whether it's luminance or chrominance data
    if isLuminance:
        DC_Huff = DC_Huffman_L
        AC_Huff = AC_Huffman_L
    else:
        DC_Huff = DC_Huffman_C
        AC_Huff = AC_Huffman_C

    # Determine DC magnitude
    Category = 0
    if runSymbols[0, 1] != 0:
        Category = int(np.floor(np.log2(np.abs(runSymbols[0, 1]))) + 1)
    DC_Magn = Category
    # Get additional bits for DC value
    dcAdditionalBits = getBinary(runSymbols[0, 1], Category)

    # Initialize arrays to store AC magnitudes and additional bits
    AC_Magn = np.zeros(RowNumber - 1, dtype=int)
    acAdditionalBits = ["" for _ in range(RowNumber - 1)]

    # Loop through the runSymbols array to determine AC magnitudes and additional bits
    for j in range(1, RowNumber):
        if np.array_equal(runSymbols[j, :], [15, 0]) or np.array_equal(runSymbols[j, :], [0, 0]):
            AC_Magn[j - 1] = 0
        else:
            Category = int(np.floor(np.log2(np.abs(runSymbols[j, 1]))) + 1)
            AC_Magn[j - 1] = Category
            acAdditionalBits[j - 1] = getBinary(runSymbols[j, 1], Category)

    # Encode DC component
    index = DC_Magn
    strListHuff = [strHuff, DC_Huff[index], dcAdditionalBits]
    strHuff = "".join(strListHuff)
    if strHuff == '000':
        strHuff ='00'

    # Encode AC components
    for i in range(1, RowNumber):
        index = runSymbols[i, 0] * 10 + AC_Magn[i - 1]

        if runSymbols[i, 0] == 15:
            index += 1
        strHuff += AC_Huff[index.astype(int)] + acAdditionalBits[i - 1]

    # Add padding bits to ensure the length of the encoded stream is a multiple of 8
    remBits = len(strHuff) % 8
    if remBits:
        addBits = "1" * (8 - remBits)
        strHuff += addBits

    # Convert the Huffman-encoded stream into bytes
    huffStream = []
    for i in range(0, len(strHuff), 8):
        huffStream.append(int(strHuff[i:i + 8], 2))

    # Convert the byte array into a decimal representation
    huffStreamDecimal = huffStream

    DecimalRepresentation = list(huffStreamDecimal)

    return huffStream

## **Huffman Decoder**

In [ ]:
def huffDec(huffStream, DC_Huff_L, DC_Huff_C, AC_Huff_L, AC_Huff_C, isLuminance):
    # Transform from bytestream to char
    strHuff = ""
    for byte in huffStream:
        currBin = format(byte, '08b')
        strHuff += currBin

    # Select Huffman tables based on whether it's luminance or chrominance data
    if isLuminance:
        DC_Huff = DC_Huff_L
        AC_Huff = AC_Huff_L
    else:
        DC_Huff = DC_Huff_C
        AC_Huff = AC_Huff_C

    totalSize = len(strHuff)
    currChar = ""
    runSymbols = []

    # Decode Huffman Code for DC Coefficient
    for i in range(totalSize):
        currChar = currChar + strHuff[i]
        index = None
        for j, code in enumerate(DC_Huff):
            if code == currChar:
                index = j
                break

        if index is not None:     # Found the category
            Category = index    #- 1

            if Category:
                currChar = strHuff[i+1:i+Category+1]  # The additional Bits
                dcAdditionalValue = getDecimal(currChar)
                endOfDC = i + len(currChar)
            else:
                dcAdditionalValue = 0   # DC Value = 0, has zero additional Value
                endOfDC = i
            runSymbols.append([0, dcAdditionalValue])
            break   # break from loop

    # Decode Huffman Code for AC Coefficients
    currChar = ""
    j = endOfDC + 1

    # Decode Huffman Code, for AC Coefficients
    # There are 4 cases:
    # i)index = 0, EOB
    # ii) index = 151, ZRL
    # iii) to be between 0 and 151, offset = 1 and
    #  iv) to be above 151, offset = 2.
    # For cases 3 and 4, i compute the remInd = (index-offset)mod10.
    # If remInd != 0, then Category = remInd and run = floor(index/10)
    # If remInd  = 0, then Category = 10     and run = floor(index/10) - 1

    while j < totalSize :
        currChar = currChar + strHuff[j]
        index = None

        for k, code in enumerate(AC_Huff):
            if code == currChar:
                index = k
                break

        if index == 0:  # EOB
            runSymbols.append([0, 0])
            break

        elif index == 151:  # ZRL
            runSymbols.append([15, 0])
            currChar = ""

        elif index is not None:
            if index > 151:
                index -= 1     # Cause of the ZRL in index = 151
            index = index - 1  # Cause of the EOB in index = 0
            remInd = (index % 10) # Remainings

            if not remInd is not None :
                Category = 10
                run = (np.floor((index) / 10)-1).astype(int)
            else:
                Category = remInd
                run = (np.floor(index / 10)).astype(int)

            currChar = strHuff[(j+1):(j+Category+2)]   # The additional Bits
            acAdditionalValue = getDecimal(currChar)
            runSymbols.append([run, acAdditionalValue])
            j += len(currChar)    # Go on to the next AC coeficient.
            currChar = ""

        j = j + 1
    return runSymbols

## **ISO Tables**

In [ ]:
def ISOTables() :
    # DC Huffman Tables
    DC_Huffman_L = ([ '00', '010', '011', 100, 101, 110, 1110, 11110, 111110, 1111110, 11111110, 111111110 ])

    DC_Huffman_C = (['00', '01', 10, 110, 1110, 11110, 111110, 1111110, 11111110, 111111110, 1111111110, 11111111110])

    # AC Huffman Tables
    AC_Huffman_L = ([1010, "00", "01", 100, 1011, 11010, 1111000, 11111000, 1111110110, 1111111110000010,
                     1111111110000011, 1100, 11011, 1111001, 111110110, 11111110110, 1111111110000100,
                     1111111110000101, 1111111110000110, 1111111110000111, 1111111110001000, 11100,
                     11111001, 1111110111, 111111110100, 1111111110001001, 1111111110001010,
                     1111111110001011, 1111111110001100, 1111111110001101, 1111111110001110,
                     111010, 111110111, 111111110101, 1111111110001111, 1111111110010000, 1111111110010001,
                     1111111110010010, 1111111110010011, 1111111110010100, 1111111110010101, 111011,
                     1111111000, 1111111110010110, 1111111110010111, 1111111110011000, 1111111110011001,
                     1111111110011010, 1111111110011011, 1111111110011100, 1111111110011101, 1111010,
                     11111110111, 1111111110011110, 1111111110011111, 1111111110100000, 1111111110100001,
                     1111111110100010, 1111111110100011, 1111111110100100, 1111111110100101, 1111011,
                     111111110110, 1111111110100110, 1111111110100111, 1111111110101000, 1111111110101001,
                     1111111110101010, 1111111110101011, 1111111110101100, 1111111110101101, 11111010,
                     111111110111, 1111111110101110, 1111111110101111, 1111111110110000, 1111111110110001,
                     1111111110110010, 1111111110110011, 1111111110110100, 1111111110110101, 111111000,
                     111111111000000, 1111111110110110, 1111111110110111, 1111111110111000,
                     1111111110111001, 1111111110111010, 1111111110111011, 1111111110111100,
                     1111111110111101, 111111001, 1111111110111110, 1111111110111111, 1111111111000000,
                     1111111111000001, 1111111111000010,  1111111111000011, 1111111111000100, 1111111111000101,
                     1111111111000110, 111111010, 1111111111000111, 1111111111001000, 1111111111001001,
                     1111111111001010, 1111111111001011, 1111111111001100, 1111111111001101, 1111111111001110,
                     1111111111001111, 1111111001, 1111111111010000, 1111111111010001, 1111111111010010,
                     1111111111010011, 1111111111010100, 1111111111010101, 1111111111010110, 1111111111010111,
                     1111111111011000, 1111111010, 1111111111011001, 1111111111011010,  1111111111011011,
                     1111111111011100, 1111111111011101, 1111111111011110, 1111111111011111, 1111111111100000,
                     1111111111100001, 11111111000, 1111111111100010, 1111111111100011, 1111111111100100,
                     1111111111100101, 1111111111100110, 1111111111100111, 1111111111101000, 1111111111101001,
                     1111111111101010, 1111111111101011, 1111111111101100, 1111111111101101, 1111111111101110,
                     1111111111101111, 1111111111110000, 1111111111110001, 1111111111110010, 1111111111110011,
                     1111111111110100, 11111111001, 1111111111110101, 1111111111110110, 1111111111110111,
                     1111111111111000, 1111111111111001, 1111111111111010, 1111111111111011, 1111111111111100,
                     1111111111111101, 1111111111111110] )

    AC_Huffman_C = (["00", "01", 100, 1010, 11000, 11001, 111000, 1111000, 111110100, 1111110110, 111111110100,
                     1011, 111001, 11110110, 111110101, 11111110110, 111111110101, 1111111110001000,
                     1111111110001001, 1111111110001010, 1111111110001011, 11010, 11110111, 1111110111,
                     111111110110, 111111111000010, 1111111110001100, 1111111110001101, 1111111110001110,
                     1111111110001111, 1111111110010000, 11011, 11111000, 1111111000, 111111110111,
                     1111111110010001, 1111111110010010, 1111111110010011, 1111111110010100,
                     1111111110010101, 1111111110010110, 111010, 111110110, 1111111110010111,
                     1111111110011000, 1111111110011001, 1111111110011010, 1111111110011011,
                     1111111110011100, 1111111110011101, 1111111110011110, 111011, 1111111001,
                     1111111110011111, 1111111110100000, 1111111110100001, 1111111110100010,
                     1111111110100011, 1111111110100100, 1111111110100101, 1111111110100110, 1111001,
                     11111110111, 1111111110100111, 1111111110101000, 1111111110101001, 1111111110101010,
                     1111111110101011, 1111111110101100, 1111111110101101, 1111111110101110, 1111010,
                     11111111000, 1111111110101111, 1111111110110000, 1111111110110001, 1111111110110010,
                     1111111110110011, 1111111110110100, 1111111110110101, 1111111110110110, 11111001,
                     1111111110110111, 1111111110111000, 1111111110111001, 1111111110111010, 1111111110111011,
                     1111111110111100, 1111111110111101, 1111111110111110, 1111111110111111, 111110111,
                     1111111111000000, 1111111111000001, 1111111111000010, 1111111111000011, 1111111111000100,
                     1111111111000101, 1111111111000110, 1111111111000111, 1111111111001000, 111111000,
                     1111111111001001, 1111111111001010, 1111111111001011, 1111111111001100, 1111111111001101,
                     1111111111001110, 1111111111001111, 1111111111010000, 1111111111010001, 111111001,
                     1111111111010010, 1111111111010011, 1111111111010100, 1111111111010101, 1111111111010110,
                     1111111111010111, 1111111111011000, 1111111111011001, 1111111111011010, 111111010,
                     1111111111011011, 1111111111011100, 1111111111011101, 1111111111011110, 1111111111011111,
                     1111111111100000, 1111111111100001, 1111111111100010, 1111111111100011, 11111111001,
                     1111111111100100, 1111111111100101, 1111111111100110, 1111111111100111, 1111111111101000,
                     1111111111101001, 1111111111101010, 1111111111101011, 1111111111101100, 11111111100000,
                     1111111111101101, 1111111111101110, 1111111111101111, 1111111111110000, 1111111111110001,
                     1111111111110010, 1111111111110011, 1111111111110100, 1111111111110101, 1111111010,
                     111111111000011, 1111111111110110, 1111111111110111, 1111111111111000, 1111111111111001,
                     1111111111111010, 1111111111111011, 1111111111111100, 1111111111111101, 1111111111111110])

     # Grouping all tables into a dictionary for easier access
    huffmanTables = {
       "DC_Huff_L": DC_Huffman_L,
       "DC_Huff_C": DC_Huffman_C,
       "AC_Huff_L": AC_Huffman_L,
       "AC_Huff_C": AC_Huffman_C
     }

    return huffmanTables


# **3.6 Demo 1**

## **Upload and Display the images**

In [ ]:
# Load the baboon image
BaboonPath = "/content/baboon.png"
imageBaboon = cv2.imread(BaboonPath)

# Get the shape of the image
height, width, channels = imageBaboon.shape
print("Baboon Image shape:", height, "x", width, "x", channels)

# Display the image
cv2_imshow(imageBaboon)

# Load the baboon image
LenaPath = "/content/lena_color_512.png"
imageLena = cv2.imread(LenaPath)

# Get the shape of the image
height, width, channels = imageLena.shape
print("\n\n Lena Color 512 Image shape:", height, "x", width, "x", channels)

# Display the image
cv2_imshow(imageLena)


## **Baboon Image**

In [ ]:
# Load the baboon image
BaboonPath = "/content/baboon.png"
imageBaboon = plt.imread(BaboonPath)

#Scale pixel values to the range [0, 255]
imageBaboon = imageBaboon* 255.0

#Convert the scaled pixel values to uint8
imageBaboon = imageBaboon.astype(np.uint8)

# Crop Baboon Image
N, M, _ = imageBaboon.shape
Nmod8 = N % 8
Mmod8 = M % 8

if Nmod8 != 0:
    NcropStart = math.floor(Nmod8/2)
    NcropEnd = round(Nmod8/2)
    imageBaboon = imageBaboon[NcropStart : -NcropEnd, :, :]
if Mmod8 != 0:
    McropStart = math.floor(Mmod8/2)
    McropEnd = round(Mmod8/2)
    imageBaboon = imageBaboon[ :, McropStart : -McropEnd, :]

# Get the shape of the image
height, width, channels = imageBaboon.shape
print("Baboon Image shape after croping border rows and columns:", height, "x", width, "x", channels)

subimg = np.array([4, 2, 2]) ;
[imageY, imageCb, imageCr] = convert2ycbcr(imageBaboon, subimg) ;
RGBimageBaboon = convert2rgb(imageY, imageCr, imageCb, subimg) ;

# Set figure size
plt.figure(figsize=(15, 5))

# Display original image
plt.subplot(1, 3, 1)
plt.imshow(imageBaboon)
plt.title('Original Image')

# Display reconstructed image
plt.subplot(1, 3, 2)
plt.imshow(RGBimageBaboon)
plt.title('Reconstructed Image - Subsampling 4:2:2')

# Display error on reconstruction
plt.subplot(1, 3, 3)
error = imageBaboon - RGBimageBaboon
plt.imshow(error)
plt.title('Error on Reconstruction')

plt.show()

In [ ]:
# Baboon image for 4:2:2 subsampling and qScale = 0.6%
[qTableL, qTableC] = changedTables(0)
subimg = np.array([4, 2, 2])
qScale = 0.6

[imageY, imageCb, imageCr] = convert2ycbcr(imageBaboon, subimg)
RowNumber, ColumnNumber = imageY.shape
imageY_rec = np.zeros_like(imageY)

for row in range(0, RowNumber, 8):
    for column in range(0, ColumnNumber, 8):
        blockY = imageY[row:row+8, column:column+8]
        dctblockY = blockDCT(blockY)
        quantblockY = quantizeJPEG(dctblockY, qTableL, qScale)
        de_quantblockY = dequantizeJPEG(quantblockY, qTableL, qScale)
        idctblockY = iBlockDCT(de_quantblockY)
        imageY_rec[row:row+8, column:column+8] = idctblockY


[RowNumber, ColumnNumber] = imageCb.shape

imageCr_rec = np.zeros_like(imageCb)
imageCb_rec = np.zeros_like(imageCb)

for row in range(0, RowNumber, 8):
    for column in range(0, ColumnNumber, 8):
        blockCr = imageCr[row:row+8, column:column+8]
        blockCb = imageCb[row:row+8, column:column+8]

        dctblockCr = blockDCT(blockCr)
        dctblockCb = blockDCT(blockCb)

        quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale)
        quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale)

        de_quantblockCr = dequantizeJPEG(quantblockCr, qTableC, qScale)
        de_quantblockCb = dequantizeJPEG(quantblockCb, qTableC, qScale)

        idctblockCr = iBlockDCT(de_quantblockCr)
        idctblockCb = iBlockDCT(de_quantblockCb)

        imageCr_rec[row:row+8, column:column+8] = idctblockCr
        imageCb_rec[row:row+8, column:column+8] = idctblockCb

# Reconstruct RGB image
imageRGB = convert2rgb(imageY_rec, imageCr_rec, imageCb_rec, subimg)

# Plotting
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(imageBaboon, cmap='viridis', interpolation='nearest')
plt.title('Original Image', fontsize=12)

plt.subplot(1, 3, 2)
plt.imshow(imageRGB)
plt.title('Reconstructed Image - Subsampling 4:2:2, qScale = 0.6', fontsize=12)

plt.subplot(1, 3, 3)
plt.imshow(imageBaboon - imageRGB)
plt.title('Quantization and Subsampling Error', fontsize=12)

plt.show()

# Calculate Mean Square Error (MSE)
MSE = np.sum((imageBaboon - imageRGB) ** 2) / imageBaboon.size
print(f"Mean Square Error for the 1st image is: {MSE}. [qScale = 0.6, Subsampling 4:2:2]")

## **Lena Image**

In [ ]:
# Load the baboon image
LenaPath = "/content/lena_color_512.png"
imageLena = plt.imread(LenaPath)

#Scale pixel values to the range [0, 255]
imageLena = imageLena* 255.0

#Convert the scaled pixel values to uint8
imageLena = imageLena.astype(np.uint8)

# Crop Baboon Image
N, M, _ = imageLena.shape
Nmod8 = N % 8
Mmod8 = M % 8

if Nmod8 != 0:
    NcropStart = math.floor(Nmod8/2)
    NcropEnd = round(Nmod8/2)
    imageLena = imageLena[NcropStart : -NcropEnd, :, :]
if Mmod8 != 0:
    McropStart = math.floor(Mmod8/2)
    McropEnd = round(Mmod8/2)
    imageLena = imageLena[ :, McropStart : -McropEnd, :]

# Get the shape of the image
height, width, channels = imageLena.shape
print("Lena Image shape after croping border rows and columns:", height, "x", width, "x", channels)

subimg = np.array([4, 4, 4]) ;
[imageY, imageCb, imageCr] = convert2ycbcr(imageLena, subimg) ;
RGBimageLena = convert2rgb(imageY, imageCr, imageCb, subimg) ;

# Set figure size
plt.figure(figsize=(15, 5))

# Display original image
plt.subplot(1, 3, 1)
plt.imshow(imageLena)
plt.title('Original Image')

# Display reconstructed image
plt.subplot(1, 3, 2)
plt.imshow(imageLena)
plt.title('Reconstructed Image - Subsampling 4:4:4')

# Display error on reconstruction
plt.subplot(1, 3, 3)
error = imageLena - RGBimageLena
plt.imshow(error)
plt.title('Error on Reconstruction')

plt.show()

In [ ]:
# Set parameters for the second image
subimg = np.array([4, 4, 4])
qScale = 5

# Convert the second image to YCbCr
[imageY, imageCb, imageCr] = convert2ycbcr(imageLena, subimg)

# Initialize arrays to store reconstructed components
imageY_rec = np.zeros_like(imageY)
imageCr_rec = np.zeros_like(imageCb)
imageCb_rec = np.zeros_like(imageCr)

# Process Y component
RowNumber, ColumnNumber = imageY.shape
for row in range(0, RowNumber, 8):
    for column in range(0, ColumnNumber, 8):
        blockY = imageY[row:row+8, column:column+8]
        dctblockY = blockDCT(blockY)
        quantblockY = quantizeJPEG(dctblockY, qTableL, qScale)
        de_quantblockY = dequantizeJPEG(quantblockY, qTableL, qScale)
        idctblockY = iBlockDCT(de_quantblockY)
        imageY_rec[row:row+8, column:column+8] = idctblockY

# Process Cb and Cr components
RowNumber, ColumnNumber = imageCb.shape
for row in range(0, RowNumber, 8):
    for column in range(0, ColumnNumber, 8):
        blockCr = imageCr[row:row+8, column:column+8]
        blockCb = imageCb[row:row+8, column:column+8]

        dctblockCr = blockDCT(blockCr)
        dctblockCb = blockDCT(blockCb)

        quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale)
        quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale)

        de_quantblockCr = dequantizeJPEG(quantblockCr, qTableC, qScale)
        de_quantblockCb = dequantizeJPEG(quantblockCb, qTableC, qScale)

        idctblockCr = iBlockDCT(de_quantblockCr)
        idctblockCb = iBlockDCT(de_quantblockCb)

        imageCr_rec[row:row+8, column:column+8] = idctblockCr
        imageCb_rec[row:row+8, column:column+8] = idctblockCb

# Reconstruct RGB image
imageRGB = convert2rgb(imageY_rec, imageCr_rec, imageCb_rec, subimg)

# Plotting
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(imageLena)
plt.title('Original Image', fontsize=12)

plt.subplot(1, 3, 2)
plt.imshow(imageRGB)
plt.title('Reconstructed Image - Subsampling 4:4:4, qScale = 5', fontsize=12)

plt.subplot(1, 3, 3)
plt.imshow(imageLena - imageRGB)
plt.title('Quantization and Subsampling Error', fontsize=12)

plt.show()

# Calculate Mean Square Error (MSE)
MSE = np.sum((imageLena - imageRGB) ** 2) / imageLena.size
print(f"Mean Square Error for the 2nd image is: {MSE}. [qScale = 5, Subsampling 4:4:4]")


# **3.7 JPEG**

## **JPEG Encoder**

In [ ]:
def findBlockIndex(Yblocks, indHor, indVer):
    for idx, block in enumerate(Yblocks):
        if block['indHor'] == indHor and block['indVer'] == indVer:
            return idx
    return None  # Return None if block not found

In [ ]:
def JPEGencode(img, subimg, qScale):
    if img.ndim != 3:
        raise ValueError("Error. 1st argument {img} must be a 3d matrix.")
    if subimg.size != 3:
        raise ValueError("Error. 2nd argument {subimg} must be a 1x3 vector.")
    if not np.isscalar(qScale):
        raise ValueError("Error. 3rd argument {qScale} must be scalar.")

    # Create the tables needed
    huffmanTables = ISOTables()
    DC_Huff_L = huffmanTables["DC_Huff_L"]
    DC_Huff_C = huffmanTables["DC_Huff_C"]
    AC_Huff_L = huffmanTables["AC_Huff_L"]
    AC_Huff_C = huffmanTables["AC_Huff_C"]

    # Convert arrays to strings
    DC_Huff_L = [str(item) for item in DC_Huff_L]
    DC_Huff_C = [str(item) for item in DC_Huff_C]
    AC_Huff_L = [str(item) for item in AC_Huff_L]
    AC_Huff_C = [str(item) for item in AC_Huff_C]

    global qTableL, qTableC
    if qTableL.size == 0:
        qTableL, qTableC = changedTables(0)
    global  isLuminance
    DCL = DC_Huff_L
    DCC = DC_Huff_C
    ACL = AC_Huff_L
    ACC = AC_Huff_C

    # Create the struct for the first element
    tableStruct = {
        'qTableL': qTableL * qScale,
        'qTableC': qTableC * qScale,
        'DCL': DCL,
        'DCC': DCC,
        'ACL': ACL,
        'ACC': ACC
    }

    JPEGenc = [tableStruct]

    # Transform to YCbCr to begin the conversion
    [imageY, imageCb, imageCr] = convert2ycbcr(img, subimg)

    # Organize the cells to appropriate order
    [RowNumber, ColumnNumber] = [imageY.shape[0], imageY.shape[1] ]
    RowNumBlock = RowNumber // 8
    ColNumBlock = ColumnNumber // 8

    # Store the Y blocks
    Yblocks = []
    idxHor = 0
    for row in range(0, RowNumber, 8):
        idxHor += 1
        idxVer = 0
        for column in range(0, ColumnNumber, 8):
            idxVer += 1

            tmpBlockY = {
                'block': imageY[row:row + 8, column:column + 8],
                'indHor': idxHor,
                'indVer': idxVer
            }
            Yblocks.append(tmpBlockY)

    MatBlocks = Yblocks
    # Determine the y index from subimg
    if not np.array_equal(subimg, [4, 2, 0]):
        y_index = list(range(1, len(Yblocks) + 1))  # Take the blocks sequentially
    else:
        y_index = []  # Take the blocks in z form
        for j in range(1, idxHor + 1, 2):
            for i in range(1, idxVer + 1, 2):
                curr_pos = (j - 1) * idxVer + i
                y_index.extend([curr_pos, curr_pos + 1, curr_pos + idxVer, curr_pos + 1 + idxVer])

    runSymMat = []

    # Create the rest of the cell
    DC_PredY = 0
    for i in y_index:
        idxRow = ((i-1) // RowNumBlock)
        idxColumn = ((i-1) % RowNumBlock)

        indexBlock = findBlockIndex(Yblocks, idxColumn+1, idxRow+1)

        blockY = Yblocks[indexBlock]['block']
        dctblockY = blockDCT(blockY)
        quantblockY = quantizeJPEG(dctblockY, qTableL, qScale)
        runSymbolsY = runLength(quantblockY, DC_PredY)
        isLuminance = 1
        huffStreamY = huffEnc(runSymbolsY, DCL, DCC, ACL, ACC, isLuminance )
        runSymMat.append(huffStreamY)
        DC_PredY = quantblockY[0, 0]  # For the next iteration

        # Save the struct for the associated block
        currStruct = {
            'blkType': 'Y',
            'indHor': Yblocks[indexBlock]['indHor'],
            'indVer': Yblocks[indexBlock]['indVer'],
            'huffStream': huffStreamY
        }
        JPEGenc.append(currStruct)

    # Encode Cb and Cr blocks
    [RowNumber, ColumnNumber] = imageCb.shape[0], imageCb.shape[1]
    DC_PredCr, DC_PredCb = 0, 0
    idxVer = 0
    for row in range(0, RowNumber, 8):
        idxVer += 1
        idxHor = 0
        for column in range(0, ColumnNumber, 8):
            idxHor += 1
            blockCr = imageCr[row:row + 8, column:column + 8]
            blockCb = imageCb[row:row + 8, column:column + 8]

            dctblockCr = blockDCT(blockCr)
            dctblockCb = blockDCT(blockCb)

            quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale)
            quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale)

            runSymbolsCr = runLength(quantblockCr, DC_PredCr)
            runSymbolsCb = runLength(quantblockCb, DC_PredCb)

            isLuminance = 0
            huffStreamCr = huffEnc(runSymbolsCr, DCL, DCC, ACL, ACC, isLuminance )
            huffStreamCb = huffEnc(runSymbolsCb, DCL, DCC, ACL, ACC, isLuminance )

            #print('Block Cr:',huffStreamCr,'\n')
            #print('Block Cb:',huffStreamCb,'\n')

            DC_PredCr = quantblockCr[0, 0]  # For the next iteration
            DC_PredCb = quantblockCb[0, 0]  # For the next iteration

            # Save the struct for the associated block (Cr)
            currStruct = {
                'blkType': 'Cr',
                'indHor': idxHor,
                'indVer': idxVer,
                'huffStream': huffStreamCr
            }
            JPEGenc.append(currStruct)

            # Save the struct for the associated block (Cb)
            currStruct = {
                'blkType': 'Cb',
                'indHor': idxHor,
                'indVer': idxVer,
                'huffStream': huffStreamCb
            }
            JPEGenc.append(currStruct)
    return JPEGenc


## **JPEG Decoder**

In [ ]:
def JPEGdecode(JPEGenc):
    if not isinstance(JPEGenc, list):
        raise ValueError("Error. Argument {JPEGenc} must be a list.")

    # Create the tables needed
    huffmanTables = ISOTables()
    DC_Huff_L = huffmanTables["DC_Huff_L"]
    DC_Huff_C = huffmanTables["DC_Huff_C"]
    AC_Huff_L = huffmanTables["AC_Huff_L"]
    AC_Huff_C = huffmanTables["AC_Huff_C"]

    # Convert arrays to strings
    DC_Huff_L = [str(item) for item in DC_Huff_L]
    DC_Huff_C = [str(item) for item in DC_Huff_C]
    AC_Huff_L = [str(item) for item in AC_Huff_L]
    AC_Huff_C = [str(item) for item in AC_Huff_C]

    global qTableL, qTableC
    if qTableL.size == 0:
        qTableL, qTableC = changedTables(0)
    global  isLuminance

    DCL = DC_Huff_L
    DCC = DC_Huff_C
    ACL = AC_Huff_L
    ACC = AC_Huff_C

    # Get the tables for the first element
    tableStruct = JPEGenc[0]
    qTableL = tableStruct['qTableL']
    qTableC = tableStruct['qTableC']

    DC_PredY = 0
    DC_PredCr = 0
    DC_PredCb = 0
    numberOfY = 0
    numberOfCb = 0
    numberOfCr = 0

    #
    # Extract all 'indHor' and 'indVer' values
    indHorValues = [block.get('indHor', 0) for block in JPEGenc]
    indVerValues = [block.get('indVer', 0) for block in JPEGenc]

    # Find the maximum value
    indHorMax = max(indHorValues)
    indVerMax = max(indVerValues)

    rowsImgY    = indVerMax * 8
    columnsImgY = indHorMax * 8

    numY  = sum(1 for block in JPEGenc if block.get('blkType') == 'Y')
    numCb = sum(1 for block in JPEGenc if block.get('blkType') == 'Cb')
    numCr = sum(1 for block in JPEGenc if block.get('blkType') == 'Cr')

    if numY == numCb and numY == numCr:
       imageY_rec  = np.zeros((rowsImgY, columnsImgY))
       imageCb_rec = np.zeros((rowsImgY, columnsImgY))
       imageCr_rec = np.zeros((rowsImgY, columnsImgY))

    elif int(numY/2) == numCb and numCr == 0 :
       imageY_rec  = np.zeros((rowsImgY, columnsImgY))
       imageCb_rec = np.zeros( (int(rowsImgY), int(columnsImgY/2)) )

    elif int(numY/2) == numCb and numCr == int(numY/2) :
       imageY_rec  = np.zeros((rowsImgY, columnsImgY))
       imageCb_rec = np.zeros( (int(rowsImgY), int(columnsImgY/2)) )
       imageCr_rec = np.zeros( (int(rowsImgY), int(columnsImgY/2)) )

    else :
       print('Wrong calculations on dimensios of the 3 channels','\n')

    for currStruct in JPEGenc[1:]:

        idxVer = currStruct['indVer']
        idxHor = currStruct['indHor']
        blkType = currStruct['blkType']

        if blkType == "Y":
            isLuminance = 1
            myrunSymbolsY = huffDec(currStruct['huffStream'], DCL, DCC, ACL, ACC, isLuminance)  # isLuminance = 1 FOR Y
            quantblockY = irunLength(myrunSymbolsY, DC_PredY)
            de_quantblockY = dequantizeJPEG(quantblockY, qTableL, 1)
            idctblockY = iBlockDCT(de_quantblockY)
            row = (idxHor - 1) * 8
            column = (idxVer - 1) * 8
            imageY_rec[row:row + 8, column:column + 8] = idctblockY
            DC_PredY = quantblockY[0, 0]  # For the next iteration
            numberOfY += 1

        elif blkType == "Cb":
            isLuminance = 0
            myrunSymbolsCb = huffDec(currStruct['huffStream'], DCL, DCC, ACL, ACC, isLuminance)  # isLuminance = 0 FOR Cb
            quantblockCb = irunLength(myrunSymbolsCb, DC_PredCb)
            de_quantblockCb = dequantizeJPEG(quantblockCb, qTableC, 1)
            idctblockCb = iBlockDCT(de_quantblockCb)
            row = (idxVer - 1) * 8
            column = (idxHor - 1) * 8
            imageCb_rec[row:row + 8, column:column + 8] = idctblockCb
            DC_PredCb = quantblockCb[0, 0]  # For the next iteration
            numberOfCb += 1

        elif blkType == "Cr":
            isLuminance = 0
            myrunSymbolsCr = huffDec(currStruct['huffStream'], DCL, DCC, ACL, ACC, isLuminance)  # isLuminance = 0 FOR Cr
            quantblockCr = irunLength(myrunSymbolsCr, DC_PredCr)
            de_quantblockCr = dequantizeJPEG(quantblockCr, qTableC, 1)
            idctblockCr = iBlockDCT(de_quantblockCr)
            row = (idxVer - 1) * 8
            column = (idxHor - 1) * 8
            imageCr_rec[row:row + 8, column:column + 8] = idctblockCr
            DC_PredCr = quantblockCr[0, 0]  # For the next iteration
            numberOfCr += 1

    if numberOfY == numberOfCb:
        subimg = np.array([4, 4, 4])
    elif numberOfY == 2 * numberOfCb:
        subimg = np.array([4, 2, 2])
    else:
        subimg = np.array([4, 2, 0])

    imgRec = convert2rgb(imageY_rec, imageCr_rec, imageCb_rec, subimg)
    return imgRec

# **Results Recording**

## **Baboon Image**

In [ ]:
# Load the baboon image
BaboonPath = "/content/baboon.png"
imageBaboon = plt.imread(BaboonPath)

#Scale pixel values to the range [0, 255]
imageBaboon = imageBaboon* 255.0

#Convert the scaled pixel values to uint8
imageBaboon = imageBaboon.astype(np.uint8)

# Crop Baboon Image
N, M, _ = imageBaboon.shape
Nmod8 = N % 8
Mmod8 = M % 8

if Nmod8 != 0:
    NcropStart = math.floor(Nmod8/2)
    NcropEnd = round(Nmod8/2)
    imageBaboon = imageBaboon[NcropStart : -NcropEnd, :, :]
if Mmod8 != 0:
    McropStart = math.floor(Mmod8/2)
    McropEnd = round(Mmod8/2)
    imageBaboon = imageBaboon[ :, McropStart : -McropEnd, :]

# Get the shape of the image
height, width, channels = imageBaboon.shape
print("Baboon Image shape after croping border rows and columns:", height, "x", width, "x", channels)

[qTableL, qTableC] = changedTables(0)
subimg = np.array([4, 4, 4])
qScale = 1

num = [20, 40, 50, 60, 63]

for i in range(len(num)):
    qTableL, qTableC = changedTables(num[i])
    JPEGenc = JPEGencode(imageBaboon, subimg, qScale)
    imgRec = JPEGdecode(JPEGenc)

    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(imageBaboon)
    plt.title('Original Image', fontsize=8)

    plt.subplot(1, 2, 2)
    plt.imshow(imgRec)
    title_str = f'Reconstructed Image - Zeroing the last {num[i]} High Frequency Coefficients.'
    plt.title(title_str, fontsize=8)

    plt.show()

    print(f'[Baboon Image] - Done zeroing the last {num[i]} High Frequency Coefficients.')

In [ ]:
subimg = np.array([4, 2, 2])
qScale = ([0.1, 0.3, 0.6, 1, 2, 5, 10])
MSE = np.zeros(len(qScale))
bitNumber = np.zeros(len(qScale))
compressRatio = np.zeros(len(qScale))

for i in range(len(qScale)):
    qTableL, qTableC = changedTables(0)
    JPEGenc = JPEGencode(imageBaboon, subimg, qScale[i])
    imgRec = JPEGdecode(JPEGenc)
    MSE[i] = np.sum((imageBaboon - imgRec) ** 2) / imageBaboon.size

    for j in range(1, len(JPEGenc)):
        currStruct = JPEGenc[j]
        bitNumber[i] += len(currStruct['huffStream']) * 8

    compressRatio[i] = (imageBaboon.size * 8) / bitNumber[i]

    plt.figure(figsize=(25, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(imageBaboon)
    plt.title('Original Image', fontsize=25)
    plt.subplot(1, 3, 2)
    plt.imshow(imgRec)
    title_str = f"Reconstructed Image - Subsampling {subimg[0]}:{subimg[1]}:{subimg[2]}, qScale = {qScale[i]}"
    plt.title(title_str, fontsize=25)
    plt.subplot(1, 3, 3)
    plt.imshow(imageBaboon - imgRec)
    plt.title('Error on reconstruction', fontsize=25)
    plt.tight_layout()

    print(f'[Baboon Image] - Done computing results for qScale = {qScale[i]:.1f}.')

plt.figure()
plt.plot(qScale, MSE, '-o')
plt.title('Baboon Image - Mean Square Error')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('MSE')
plt.grid(True)

plt.figure()
plt.plot(qScale, compressRatio, '-o')
plt.title('Baboon Image - Compression Ratio')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('Compression Ratio')
plt.grid(True)

plt.figure()
plt.plot(qScale, bitNumber, '-o')
plt.title('Baboon Image - Number of bits [Encoded Image]')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('Number of bits')
plt.grid(True)

plt.figure()
plt.plot(bitNumber, MSE, '-o')
plt.title('Baboon Image - Mean Square Error and Number of bits')
plt.xlabel('Number of bits')
plt.ylabel('MSE')
plt.grid(True)

plt.show()

## **Lena Image**

In [ ]:
# Load the Lena image
LenaPath = "/content/lena_color_512.png"
imageLena = plt.imread(LenaPath)

#Scale pixel values to the range [0, 255]
imageLena = imageLena* 255.0

#Convert the scaled pixel values to uint8
imageLena = imageLena.astype(np.uint8)

# Crop Lena Image
N, M, _ = imageLena.shape
Nmod8 = N % 8
Mmod8 = M % 8

if Nmod8 != 0:
    NcropStart = math.floor(Nmod8/2)
    NcropEnd = round(Nmod8/2)
    imageLena = imageLena[NcropStart : -NcropEnd, :, :]
if Mmod8 != 0:
    McropStart = math.floor(Mmod8/2)
    McropEnd = round(Mmod8/2)
    imageLena = imageLena[ :, McropStart : -McropEnd, :]

# Get the shape of the image
height, width, channels = imageLena.shape
print("Lena Image shape after croping border rows and columns:", height, "x", width, "x", channels)

[qTableL, qTableC] = changedTables(0)
subimg = np.array([4, 4, 4])
qScale = 1

num = [20, 40, 50, 60, 63]

for i in range(len(num)):
    qTableL, qTableC = changedTables(num[i])
    JPEGenc = JPEGencode(imageLena, subimg, qScale)
    imgRec = JPEGdecode(JPEGenc)

    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(imageLena)
    plt.title('Original Image', fontsize=8)

    plt.subplot(1, 2, 2)
    plt.imshow(imgRec)
    title_str = f'Reconstructed Image - Zeroing the last {num[i]} High Frequency Coefficients.'
    plt.title(title_str, fontsize=8)

    plt.show()

    print(f'[Lena Image] - Done zeroing the last {num[i]} High Frequency Coefficients.')

In [ ]:
subimg = np.array([4, 4, 4])
qScale = ([0.1, 0.3, 0.6, 1, 2, 5, 10])
MSE = np.zeros(len(qScale))
bitNumber = np.zeros(len(qScale))
compressRatio = np.zeros(len(qScale))

for i in range(len(qScale)):
    qTableL, qTableC = changedTables(0)
    JPEGenc = JPEGencode(imageLena, subimg, qScale[i])
    imgRec = JPEGdecode(JPEGenc)
    MSE[i] = np.sum((imageLena - imgRec) ** 2) / imageLena.size

    for j in range(1, len(JPEGenc)):
        currStruct = JPEGenc[j]
        bitNumber[i] += len(currStruct['huffStream']) * 8

    compressRatio[i] = (imageLena.size * 8) / bitNumber[i]
    plt.figure(figsize=(25, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(imageLena)
    plt.title('Original Image', fontsize=25)
    plt.subplot(1, 3, 2)
    plt.imshow(imgRec)
    title_str = f"Reconstructed Image - Subsampling {subimg[0]}:{subimg[1]}:{subimg[2]}, qScale = {qScale[i]}"
    plt.title(title_str, fontsize=25)
    plt.subplot(1, 3, 3)
    plt.imshow(imageLena - imgRec)
    plt.title('Error on reconstruction', fontsize=25)
    plt.tight_layout()

    print(f'[Lena Image] - Done computing results for qScale = {qScale[i]:.1f}.')

plt.figure()
plt.plot(qScale, MSE, '-o')
plt.title('Lena Image - Mean Square Error')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('MSE')
plt.grid(True)

plt.figure()
plt.plot(qScale, compressRatio, '-o')
plt.title('Lena Image - Compression Ratio')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('Compression Ratio')
plt.grid(True)

plt.figure()
plt.plot(qScale, bitNumber, '-o')
plt.title('Lena Image - Number of bits [Encoded Image]')
plt.xticks(qScale,fontsize=5)
plt.xlabel('qScale')
plt.ylabel('Number of bits')
plt.grid(True)

plt.figure()
plt.plot(bitNumber, MSE, '-o')
plt.title('Lena Image - Mean Square Error and Number of bits')
plt.xlabel('Number of bits')
plt.ylabel('MSE')
plt.grid(True)

plt.show()

# **Demo 2**

## **Baboon Image**

In [ ]:
# For the Y component
subimg = np.array([4, 2, 2])
qScale = 0.6
imageY, imageCb, imageCr = convert2ycbcr(imageBaboon, subimg)
RowNumberY, ColumnNumberY = imageY.shape
allQuantY = []
allRunlengthY = []
DC_PredY = 0

for row in range(0, RowNumberY, 8):
    for column in range(0, ColumnNumberY, 8):
        qTableL, qTableC = changedTables(0)
        blockY = imageY[row:row + 8, column:column + 8]
        dctblockY = blockDCT(blockY)
        quantblockY = quantizeJPEG(dctblockY, qTableL, qScale)
        runSymbolsY = runLength(quantblockY, DC_PredY)
        DC_PredY = quantblockY[0, 0]  # For the next iteration

        allRunlengthY.append(runSymbolsY)
        allQuantY.append(quantblockY)

allRunlengthY = np.concatenate(allRunlengthY, axis=0)
allQuantY = np.concatenate(allQuantY, axis=1)


RowNumberCbCr, ColumnNumberCbCr = imageCb.shape
allQuantCb = []
allQuantCr = []
allRunlengthCb = []
allRunlengthCr = []
DC_PredCr = 0
DC_PredCb = 0

for row in range(0, RowNumberCbCr, 8):
    for column in range(0, ColumnNumberCbCr, 8):
        qTableL, qTableC = changedTables(0)
        blockCr = imageCr[row:row + 8, column:column + 8]
        blockCb = imageCb[row:row + 8, column:column + 8]

        dctblockCr = blockDCT(blockCr)
        dctblockCb = blockDCT(blockCb)

        quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale)
        quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale)

        runSymbolsCr = runLength(quantblockCr, DC_PredCr)
        runSymbolsCb = runLength(quantblockCb, DC_PredCb)

        DC_PredCr = quantblockCr[0, 0]  # For the next iteration
        DC_PredCb = quantblockCb[0, 0]  # For the next iteration

        allRunlengthCb.append(runSymbolsCb)
        allRunlengthCr.append(runSymbolsCr)
        allQuantCb.append(quantblockCb)
        allQuantCr.append(quantblockCr)

allRunlengthCb = np.concatenate(allRunlengthCb, axis=0)
allRunlengthCr = np.concatenate(allRunlengthCr, axis=0)

allQuantCb = np.concatenate(allQuantCb, axis=1)
allQuantCr = np.concatenate(allQuantCr, axis=1)


red = imageBaboon[:,:,0]
green = imageBaboon[:,:,1]
blue = imageBaboon[:,:,2]

# Compute entropy of Spatial Domain
_, icRed = np.unique(red, return_inverse=True)
_, icGreen = np.unique(green, return_inverse=True)
_, icBlue = np.unique(blue, return_inverse=True)

# Count the freq of the quantized values
freqRed = np.bincount(icRed) / red.size
freqGreen = np.bincount(icGreen) / green.size
freqBlue = np.bincount(icBlue) / blue.size

# Entropy is -freq*log(freq)
entropySpatialAll = np.sum(-freqRed * np.log2(freqRed)) + np.sum(-freqGreen * np.log2(freqGreen)) + \
                      np.sum(-freqBlue * np.log2(freqBlue))
print('[First Image] ~ The entropy for the Spatial Domain is:', entropySpatialAll, '[per symbol].')
print('[First Image] ~ The entropy for the Spatial Domain is:', entropySpatialAll * imageBaboon.shape[0]*imageBaboon.shape[1], '.')

# Compute entropy for the quantized values
_, icQuantY = np.unique(allQuantY, return_inverse=True)
_, icQuantCb = np.unique(allQuantCb, return_inverse=True)
_, icQuantCr = np.unique(allQuantCr, return_inverse=True)

# Count the freq of the quantized values
freqQuantY = np.bincount(icQuantY) / allQuantY.size
freqQuantCb = np.bincount(icQuantCb) / allQuantCb.size
freqQuantCr = np.bincount(icQuantCr) / allQuantCr.size

# Entropy is -freq*log(freq)
entropyQuantAll_perSymbol = np.sum(-freqQuantY * np.log2(freqQuantY)) + np.sum(-freqQuantCb * np.log2(freqQuantCb)) + \
                            np.sum(-freqQuantCr * np.log2(freqQuantCr))
print('\n[First Image] ~ The entropy for the Quantize DCT Coefficients is:', entropyQuantAll_perSymbol, '[per symbol].')
entropyQuantAll = np.sum(-freqQuantY * np.log2(freqQuantY)) * (RowNumberY * ColumnNumberY) + \
                  np.sum(-freqQuantCb * np.log2(freqQuantCb)) * (RowNumberCbCr * ColumnNumberCbCr) + \
                  np.sum(-freqQuantCr * np.log2(freqQuantCr)) * (RowNumberCbCr * ColumnNumberCbCr)
print('[First Image] ~ The entropy for the Quantize DCT Coefficients is:', entropyQuantAll, '.')

# Compute entropy for runlength
allRunlengths = np.concatenate((allRunlengthY, allRunlengthCb, allRunlengthCr))
# Find the unique rows
_, icRuns = np.unique(allRunlengths[:,0:2], axis=0, return_inverse=True)
# Count the freq of those rows in the runlengths
freqRun = np.bincount(icRuns) / len(allRunlengths)
# Entropy is -freq*log(freq)
entropyRunlength_perSymbol = np.sum(-freqRun * np.log2(freqRun))
print('\n[First Image] ~ The entropy for the Runlength is:', entropyRunlength_perSymbol, '[per Symbol]')
print('[First Image] ~ The entropy for the Runlength is:', entropyRunlength_perSymbol * len(allRunlengths), '.')


In [ ]:
subimg = np.array([[4, 4, 4],
                   [4, 2, 2],
                   [4, 2, 0]])

qScale = np.array([0.1, 0.3, 0.6, 1, 2, 5, 10])
entropyRunlength = np.zeros((len(subimg),len(qScale)))
# Image 1
for i in range(len(subimg)):  # for each resolution
    curr_subimg = subimg[i,:]

    for j in range(len(qScale)):

        imageY, imageCb, imageCr = convert2ycbcr(imageBaboon, curr_subimg)
        RowNumberY, ColumnNumberY = imageY.shape
        allQuantY = []
        allRunlengthY = []
        DC_PredY = 0
        for row in range(0, RowNumberY, 8):
            for column in range(0, ColumnNumberY, 8):
                qTableL, qTableC = changedTables(0)
                blockY = imageY[row:row + 8, column:column + 8]
                dctblockY = blockDCT(blockY)
                quantblockY = quantizeJPEG(dctblockY, qTableL, qScale[j])
                runSymbolsY = runLength(quantblockY, DC_PredY)

                DC_PredY = quantblockY[0, 0]  # For the next iteration
                allRunlengthY.append(runSymbolsY)
                allQuantY.append(quantblockY)

        allRunlengthY = np.concatenate(allRunlengthY, axis=0)
        allQuantY = np.concatenate(allQuantY, axis=1)

        # For the Cb, Cr components
        RowNumberCbCr, ColumnNumberCbCr = imageCb.shape
        allQuantCb = []
        allQuantCr = []
        allRunlengthCb = []
        allRunlengthCr = []
        DC_PredCr = 0
        DC_PredCb = 0

        for row in range(0, RowNumberCbCr, 8):
            for column in range(0, ColumnNumberCbCr, 8):
                qTableL, qTableC = changedTables(0)
                blockCr = imageCr[row:row + 8, column:column + 8]
                blockCb = imageCb[row:row + 8, column:column + 8]

                dctblockCr = blockDCT(blockCr)
                dctblockCb = blockDCT(blockCb)

                quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale[j])
                quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale[j])

                runSymbolsCr = runLength(quantblockCr, DC_PredCr)
                runSymbolsCb = runLength(quantblockCb, DC_PredCb)

                DC_PredCr = quantblockCr[0, 0]  # For the next iteration
                DC_PredCb = quantblockCb[0, 0]  # For the next iteration

                allRunlengthCb.append(runSymbolsCb)
                allRunlengthCr.append(runSymbolsCr)

                allQuantCb.append(quantblockCb)
                allQuantCr.append(quantblockCr)

        allRunlengthCb = np.concatenate(allRunlengthCb, axis=0)
        allRunlengthCr = np.concatenate(allRunlengthCr, axis=0)

        allQuantCb = np.concatenate(allQuantCb, axis=1)
        allQuantCr = np.concatenate(allQuantCr, axis=1)

        # Compute entropy for runlength
        allRunlengths = np.vstack((allRunlengthY, allRunlengthCb, allRunlengthCr))
        _, icRuns = np.unique(allRunlengths[:, :2], axis=0, return_inverse=True)
        freqRun = np.bincount(icRuns) / len(allRunlengths)
        entropyRunlength[i, j] = np.sum(-freqRun * np.log2(freqRun)) * len(allRunlengths)

        print(f"[First Image] ~ The entropy for the Runlength is: {entropyRunlength[i, j]:e}. [SubSampling {curr_subimg[0]}:{curr_subimg[1]}:{curr_subimg[2]}], [qScale = {qScale[j]:.2f}]")

    plt.figure()
    plt.plot(qScale, entropyRunlength[i], '-o')
    title_str = f"Entropy of Runlegths [SubSampling {curr_subimg[0]}:{curr_subimg[1]}:{curr_subimg[2]}] [First Image]"
    plt.title(title_str, fontsize=12)
    plt.xlabel('qScale', fontsize=12)
    plt.ylabel('Entropy', fontsize=12)
    plt.grid(True)
    plt.show()



## **Lena Image**

In [ ]:
# For the Y component
subimg = np.array([4, 2, 2])
qScale = 0.6
imageY, imageCb, imageCr = convert2ycbcr(imageBaboon, subimg)
RowNumberY, ColumnNumberY = imageY.shape
allQuantY = []
allRunlengthY = []
DC_PredY = 0

for row in range(0, RowNumberY, 8):
    for column in range(0, ColumnNumberY, 8):
        qTableL, qTableC = changedTables(0)
        blockY = imageY[row:row + 8, column:column + 8]
        dctblockY = blockDCT(blockY)
        quantblockY = quantizeJPEG(dctblockY, qTableL, qScale)
        runSymbolsY = runLength(quantblockY, DC_PredY)
        DC_PredY = quantblockY[0, 0]  # For the next iteration

        allRunlengthY.append(runSymbolsY)
        allQuantY.append(quantblockY)

allRunlengthY = np.concatenate(allRunlengthY, axis=0)
allQuantY = np.concatenate(allQuantY, axis=1)


RowNumberCbCr, ColumnNumberCbCr = imageCb.shape
allQuantCb = []
allQuantCr = []
allRunlengthCb = []
allRunlengthCr = []
DC_PredCr = 0
DC_PredCb = 0

for row in range(0, RowNumberCbCr, 8):
    for column in range(0, ColumnNumberCbCr, 8):
        qTableL, qTableC = changedTables(0)
        blockCr = imageCr[row:row + 8, column:column + 8]
        blockCb = imageCb[row:row + 8, column:column + 8]

        dctblockCr = blockDCT(blockCr)
        dctblockCb = blockDCT(blockCb)

        quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale)
        quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale)

        runSymbolsCr = runLength(quantblockCr, DC_PredCr)
        runSymbolsCb = runLength(quantblockCb, DC_PredCb)

        DC_PredCr = quantblockCr[0, 0]  # For the next iteration
        DC_PredCb = quantblockCb[0, 0]  # For the next iteration

        allRunlengthCb.append(runSymbolsCb)
        allRunlengthCr.append(runSymbolsCr)
        allQuantCb.append(quantblockCb)
        allQuantCr.append(quantblockCr)

allRunlengthCb = np.concatenate(allRunlengthCb, axis=0)
allRunlengthCr = np.concatenate(allRunlengthCr, axis=0)

allQuantCb = np.concatenate(allQuantCb, axis=1)
allQuantCr = np.concatenate(allQuantCr, axis=1)


red = imageBaboon[:,:,0]
green = imageBaboon[:,:,1]
blue = imageBaboon[:,:,2]

# Compute entropy of Spatial Domain
_, icRed = np.unique(red, return_inverse=True)
_, icGreen = np.unique(green, return_inverse=True)
_, icBlue = np.unique(blue, return_inverse=True)

# Count the freq of the quantized values
freqRed = np.bincount(icRed) / red.size
freqGreen = np.bincount(icGreen) / green.size
freqBlue = np.bincount(icBlue) / blue.size

# Entropy is -freq*log(freq)
entropySpatialAll = np.sum(-freqRed * np.log2(freqRed)) + np.sum(-freqGreen * np.log2(freqGreen)) + \
                      np.sum(-freqBlue * np.log2(freqBlue))
print('[First Image] ~ The entropy for the Spatial Domain is:', entropySpatialAll, '[per symbol].')
print('[First Image] ~ The entropy for the Spatial Domain is:', entropySpatialAll * imageBaboon.shape[0]*imageBaboon.shape[1], '.')

# Compute entropy for the quantized values
_, icQuantY = np.unique(allQuantY, return_inverse=True)
_, icQuantCb = np.unique(allQuantCb, return_inverse=True)
_, icQuantCr = np.unique(allQuantCr, return_inverse=True)

# Count the freq of the quantized values
freqQuantY = np.bincount(icQuantY) / allQuantY.size
freqQuantCb = np.bincount(icQuantCb) / allQuantCb.size
freqQuantCr = np.bincount(icQuantCr) / allQuantCr.size

# Entropy is -freq*log(freq)
entropyQuantAll_perSymbol = np.sum(-freqQuantY * np.log2(freqQuantY)) + np.sum(-freqQuantCb * np.log2(freqQuantCb)) + \
                            np.sum(-freqQuantCr * np.log2(freqQuantCr))
print('\n[First Image] ~ The entropy for the Quantize DCT Coefficients is:', entropyQuantAll_perSymbol, '[per symbol].')
entropyQuantAll = np.sum(-freqQuantY * np.log2(freqQuantY)) * (RowNumberY * ColumnNumberY) + \
                  np.sum(-freqQuantCb * np.log2(freqQuantCb)) * (RowNumberCbCr * ColumnNumberCbCr) + \
                  np.sum(-freqQuantCr * np.log2(freqQuantCr)) * (RowNumberCbCr * ColumnNumberCbCr)
print('[First Image] ~ The entropy for the Quantize DCT Coefficients is:', entropyQuantAll, '.')

# Compute entropy for runlength
allRunlengths = np.concatenate((allRunlengthY, allRunlengthCb, allRunlengthCr))
# Find the unique rows
_, icRuns = np.unique(allRunlengths[:,0:2], axis=0, return_inverse=True)
# Count the freq of those rows in the runlengths
freqRun = np.bincount(icRuns) / len(allRunlengths)
# Entropy is -freq*log(freq)
entropyRunlength_perSymbol = np.sum(-freqRun * np.log2(freqRun))
print('\n[First Image] ~ The entropy for the Runlength is:', entropyRunlength_perSymbol, '[per Symbol]')
print('[First Image] ~ The entropy for the Runlength is:', entropyRunlength_perSymbol * len(allRunlengths), '.')


In [ ]:
subimg = np.array([[4, 4, 4],
                   [4, 2, 2],
                   [4, 2, 0]])

qScale = np.array([0.1, 0.3, 0.6, 1, 2, 5, 10])
entropyRunlength = np.zeros((len(subimg),len(qScale)))
# Image 1
for i in range(len(subimg)):  # for each resolution
    curr_subimg = subimg[i,:]

    for j in range(len(qScale)):

        imageY, imageCb, imageCr = convert2ycbcr(imageBaboon, curr_subimg)
        RowNumberY, ColumnNumberY = imageY.shape
        allQuantY = []
        allRunlengthY = []
        DC_PredY = 0
        for row in range(0, RowNumberY, 8):
            for column in range(0, ColumnNumberY, 8):
                qTableL, qTableC = changedTables(0)
                blockY = imageY[row:row + 8, column:column + 8]
                dctblockY = blockDCT(blockY)
                quantblockY = quantizeJPEG(dctblockY, qTableL, qScale[j])
                runSymbolsY = runLength(quantblockY, DC_PredY)

                DC_PredY = quantblockY[0, 0]  # For the next iteration
                allRunlengthY.append(runSymbolsY)
                allQuantY.append(quantblockY)

        allRunlengthY = np.concatenate(allRunlengthY, axis=0)
        allQuantY = np.concatenate(allQuantY, axis=1)

        # For the Cb, Cr components
        RowNumberCbCr, ColumnNumberCbCr = imageCb.shape
        allQuantCb = []
        allQuantCr = []
        allRunlengthCb = []
        allRunlengthCr = []
        DC_PredCr = 0
        DC_PredCb = 0

        for row in range(0, RowNumberCbCr, 8):
            for column in range(0, ColumnNumberCbCr, 8):
                qTableL, qTableC = changedTables(0)
                blockCr = imageCr[row:row + 8, column:column + 8]
                blockCb = imageCb[row:row + 8, column:column + 8]

                dctblockCr = blockDCT(blockCr)
                dctblockCb = blockDCT(blockCb)

                quantblockCr = quantizeJPEG(dctblockCr, qTableC, qScale[j])
                quantblockCb = quantizeJPEG(dctblockCb, qTableC, qScale[j])

                runSymbolsCr = runLength(quantblockCr, DC_PredCr)
                runSymbolsCb = runLength(quantblockCb, DC_PredCb)

                DC_PredCr = quantblockCr[0, 0]  # For the next iteration
                DC_PredCb = quantblockCb[0, 0]  # For the next iteration

                allRunlengthCb.append(runSymbolsCb)
                allRunlengthCr.append(runSymbolsCr)

                allQuantCb.append(quantblockCb)
                allQuantCr.append(quantblockCr)

        allRunlengthCb = np.concatenate(allRunlengthCb, axis=0)
        allRunlengthCr = np.concatenate(allRunlengthCr, axis=0)

        allQuantCb = np.concatenate(allQuantCb, axis=1)
        allQuantCr = np.concatenate(allQuantCr, axis=1)

        # Compute entropy for runlength
        allRunlengths = np.vstack((allRunlengthY, allRunlengthCb, allRunlengthCr))
        _, icRuns = np.unique(allRunlengths[:, :2], axis=0, return_inverse=True)
        freqRun = np.bincount(icRuns) / len(allRunlengths)
        entropyRunlength[i, j] = np.sum(-freqRun * np.log2(freqRun)) * len(allRunlengths)

        print(f"[First Image] ~ The entropy for the Runlength is: {entropyRunlength[i, j]:e}. [SubSampling {curr_subimg[0]}:{curr_subimg[1]}:{curr_subimg[2]}], [qScale = {qScale[j]:.2f}]")

    plt.figure()
    plt.plot(qScale, entropyRunlength[i], '-o')
    title_str = f"Entropy of Runlegths [SubSampling {curr_subimg[0]}:{curr_subimg[1]}:{curr_subimg[2]}] [First Image]"
    plt.title(title_str, fontsize=12)
    plt.xlabel('qScale', fontsize=12)
    plt.ylabel('Entropy', fontsize=12)
    plt.grid(True)
    plt.show()

